<a href="https://colab.research.google.com/github/sayarghoshroy/LoClus/blob/main/cluster_neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import math
import tqdm
import json
import pandas as pd

In [2]:
sys.setrecursionlimit(int(1e7))

In [3]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [4]:
path = 'drive/MyDrive/place2crash_data/'

with open(path + 'train_data.json', 'r+') as f:
  train_data = json.load(f)

with open(path + 'test_data.json', 'r+') as f:
  test_data = json.load(f)

with open(path + 'head_to_col_id.json', 'r+') as f:
  mapping = json.load(f)

raw_data = train_data + test_data
# Not utilizing price labels

In [5]:
# Uncomment to view the Feature name to ID mapping:
# mapping

In [6]:
# Raw data format:
print(raw_data[0])

[30913224, 'Cozy and Sunny Room Williamsburg, Luxury Building', 'Brooklyn', 'Williamsburg', 40.70959, -73.94652, 'Private room', 80, 3, 2, 0.31, 0]


In [7]:
data = []
size = 0

for item in raw_data:
  point = {}
  point['id'] = item[0]
  point['representative_point'] = {'latitude': float(item[4]),
                                    'longitude': float(item[5])}
  point['neighborhood'] = item[3]
  point['room_type'] = item[6]
  
  data.append(point)
            
size = len(data)

In [8]:
# Verifying correctness
print('First Datapoint:')
print(data[0])

First Datapoint:
{'id': 30913224, 'representative_point': {'latitude': 40.70959, 'longitude': -73.94652}, 'neighborhood': 'Williamsburg', 'room_type': 'Private room'}


In [9]:
# Number of items
print('Number of Points: ' + str(len(data)))

Number of Points: 48895


In [10]:
# Categorizing points

argument = 'neighborhood'

category_dict = {}
valid_datapoints = 0

for item in data:
    if argument not in item.keys():
        continue
    if item[argument] not in category_dict.keys():
        category_dict[item[argument]] = []

    category_dict[item[argument]].append(item)
    valid_datapoints += 1
    
print('Number of datapoints having ' + argument + ' = ' + str(valid_datapoints))

Number of datapoints having neighborhood = 48895


In [11]:
print('Number of categories: ' + str(len(category_dict.keys())))
stats = []

for key in category_dict.keys():
    stats.append([key, len(category_dict[key])])
    
df = pd.DataFrame(stats, columns = [argument, 'count'])
df.sort_values(by = 'count', inplace = True, ascending = False)
df.reset_index(drop = True, inplace = True)
display(df)
df.to_csv(path + 'results/' + argument + '-wise-stats.csv', index = False)

Number of categories: 221


,neighborhood,count
0,Williamsburg,3920
1,Bedford-Stuyvesant,3714
2,Harlem,2658
3,Bushwick,2465
4,Upper West Side,1971
...,...,...
216,Richmondtown,1
217,Willowbrook,1
218,Rossville,1
219,New Dorp,1


In [12]:
def comp_distance(A, B):
    lat_A = math.radians(A[0])
    lat_B = math.radians(B[0])
    long_A = math.radians(A[1])
    long_B = math.radians(B[1])
    
    diff_lat = lat_B - lat_A
    diff_long = long_B - long_A
    
    step_A = math.sin(diff_lat / 2) ** 2
    step_B = math.cos(lat_A) * math.cos(lat_B) * math.sin(diff_long / 2) ** 2
    
    comp = 2 * math.asin(math.sqrt(step_A + step_B))
    radius = 6.3781 * 1e6
    # Radius of Earth in meters
    distance = radius * comp
    
    # Returns distance in meters
    return distance

def get_point(point_struct):
    try:
        lat = point_struct['latitude']
        lon = point_struct['longitude']
    
    except Exception as E:
        return [0, 0]
    
    return [lat, lon]

def get_distance(point_struct_A, point_struct_B):
    return comp_distance(get_point(point_struct_A), get_point(point_struct_B))

def get_point_distance(item_A, item_B):
    point_struct_A = item_A['representative_point']
    point_struct_B = item_B['representative_point']
    
    return get_distance(point_struct_A, point_struct_B)

# Testing
print('comp_distance([53.32, -1.73], [53.321, -1.69]): '
      + str(comp_distance([53.32, -1.73], [53.321, -1.69]))
      + ' meters.')

print('get_point_distance(data[0], data[1]): ' + str(get_point_distance(data[0], data[1])) + ' meters.')

comp_distance([53.32, -1.73], [53.321, -1.69]): 2662.128538636821 meters.
get_point_distance(data[0], data[1]): 9487.843744689368 meters.


In [13]:
# To run clustering for a particular neighborhood
argument == 'neighborhood'

# # Enter neighborhood
neighborhood_name = 'Harlem'

items = category_dict[neighborhood_name]

In [14]:
# To run for all neighborhoods
# neighborhood = 'all'
# items = data

In [15]:
threshold = 50
# Set a threshold distance in meters

In [16]:
# Testing out pairwise distance computation

# Mark True for testing:
check_distances = False

if check_distances == True:
    
    distances = []
    count = 0

    print('Neighborhood: ' + str(neighborhood_name), flush = True)

    for item_A in tqdm.tqdm(items):
        for item_B in items:
            if item_A['id'] == item_B['id']:
                continue
            else:
                distances.append(get_point_distance(item_A, item_B))
                if distances[-1] <= threshold:
                    count += 1

    print('Total number of distances: ' + str(len(distances)), flush = True)
    print('Number of pairwise distances <= ' + str(threshold) + ' meters : ' + str(count), flush = True)

In [17]:
# Preparing global data structures

map_items = {}
adj_items = {}

def initialize():
    global map_items, item_adj
    map_items = {}
    adj_items = {}
    return
    
def build_map():
    global items, map_items
    for item in items:
        map_items[item['id']] = item
    return

def build_adj():
    global items, adj_items
    
    for key in map_items.keys():
        adj_items[key] = []
    
    for item_A in tqdm.tqdm(items):
        for item_B in items:
            if item_A['id'] == item_B['id']:
                continue
            else:
                distance = get_point_distance(item_A, item_B)
                if distance <= threshold:
                    # Creating the adjacency lists
                    adj_items[item_A['id']].append(item_B['id'])
    return

# Uncomment for testing:
# initialize()
# build_map()
# build_adj()

In [18]:
# Checking correctness of adjacency list creation
check = True

if check == True:
    for key in adj_items.keys():
        if len(adj_items[key]) < 4:
            continue
        
        print(str(key) + str(': '))
        print(adj_items[key])
        break

In [19]:
degree_items = {}
cluster_items = {}
cluster_to_size_map = {}

def compute_degrees():
    global adj_items, degree_items
    for key in adj_items.keys():
        degree_items[key] = len(adj_items[key])
    return
        
cluster_ID = 0

def dft(item_ID):
    # Runs a depth first traversal of the graph from item_ID
    # Marks items with the present cluster ID
    global adj_items, cluster_items, cluster_ID
    
    if item_ID in cluster_items.keys():
        return
    
    cluster_items[item_ID] = cluster_ID
    
    for link in adj_items[item_ID]:
        dft(link)
    
    return
    
def connected_components():
    global map_items, adj_items, cluster_items, cluster_ID
    display = False
    
    cluster_ID = 0
    
    for node in map_items.keys():
        if node in cluster_items.keys():
            continue
        else:
            cluster_ID += 1
            dft(node)
    
    if display == True:
        print('Number of Clusters = ' + str(cluster_ID))
    return

def map_cluster_to_size():
    global cluster_items, cluster_to_size_map
    
    for item in cluster_items.keys():
        cluster_ID = cluster_items[item]
        
        if cluster_ID not in cluster_to_size_map.keys():
            cluster_to_size_map[cluster_ID] = 1
        else:
            cluster_to_size_map[cluster_ID] += 1
            
    return

# Uncomment for testing:
# compute_degrees()
# connected_components()
# map_cluster_to_size()

In [20]:
all_neighborhood_info = []
save_jsons = True

def store():
    global degree_items, cluster_items, map_items, save_jsons, cluster_to_size_map
    global neighborhood_name, threshold, all_neighborhood_info
    
    display_df = False
    item_info = []
    
    for key in map_items.keys():
        map_items[key]['degree'] = degree_items[key]
        map_items[key]['cluster'] = cluster_items[key]
        
        cluster_ID = cluster_items[key]
        cluster_size = cluster_to_size_map[cluster_ID]
        
        item_info.append([key, cluster_items[key], degree_items[key], 
                          map_items[key]['representative_point']['latitude'], 
                          map_items[key]['representative_point']['longitude'], 
                          map_items[key]['neighborhood'], cluster_size,
                          str(str(cluster_items[key]) + '_' + map_items[key]['neighborhood'])])
    
    all_neighborhood_info += item_info
    if save_jsons:
        with open(path + 'results/' + argument + '-' + neighborhood_name + '-' + str(threshold) + '-map_cluster_info.json', 'w+') as f:
            json.dump(map_items, f)
    
    df = pd.DataFrame(item_info, columns = ['item ID', 'Cluster ID', 'Degree', 'latitude', 'longitude', 'Neighborhood', 'Cluster_Size', 'ID_Neighborhood'])
    df.sort_values(['Neighborhood', 'Cluster_Size', 'Cluster ID', 'Degree'], ascending = (True, False, True, False), inplace = True)
    df.reset_index(drop = True, inplace = True)
    
    if display_df == True:
        display(df)
    
    df.to_csv(path + 'results/' + argument + '-' + neighborhood_name + '-' + str(threshold) + '-cluster_info.csv', index = False)
    return
    
# Uncomment for testing:
# store()

In [21]:
all_neighborhood_info = []

hood_count = int(1e3)
consider_limit = int(1e4)

for serial, neighborhood_name in enumerate(category_dict.keys()):
    if serial > hood_count:
        break

    items = category_dict[neighborhood_name]
    
    if len(items) > consider_limit:
        continue
    
    tqdm.tqdm.write('Neighborhood: ' + neighborhood_name)

    map_items = {}
    adj_items = {}
    
    initialize()
    build_map()
    build_adj()

    degree_items = {}
    cluster_items = {}
    cluster_to_size_map = {}
    
    compute_degrees()
    connected_components()
    map_cluster_to_size()

    store()
    tqdm.tqdm.write('')

Neighborhood: Williamsburg


100%|██████████| 3920/3920 [00:52<00:00, 74.94it/s]



Neighborhood: Upper West Side


100%|██████████| 1971/1971 [00:11<00:00, 170.28it/s]



Neighborhood: Clinton Hill


...

100%|██████████| 4/4 [00:00<00:00, 19328.59it/s]



Neighborhood: Fort Wadsworth


100%|██████████| 1/1 [00:00<00:00, 6831.11it/s]



Neighborhood: Howland Hook


100%|██████████| 2/2 [00:00<00:00, 13189.64it/s]


In [22]:
# All neighborhood details in one files

if save_jsons:
    with open(path + 'results/' + 'all_neighborhood_wise' + '-' + str(threshold) + '-map_cluster_info.json', 'w+') as f:
        json.dump(map_items, f)

df = pd.DataFrame(all_neighborhood_info, columns = ['item ID', 'Cluster ID', 'Degree', 'latitude', 'longitude', 'Neighborhood', 'Cluster_Size', 'ID_Neighborhood'])
df.sort_values(['Cluster_Size', 'Cluster ID', 'Degree', 'ID_Neighborhood'], ascending = (False, False, False, True), inplace = True)
df.reset_index(drop = True, inplace = True)

display(df)

df.to_csv(path + 'results/' + 'all_neighborhood_wise' + '-' + str(threshold) + '-cluster_info.csv', index = False)

,item ID,Cluster ID,Degree,latitude,longitude,Neighborhood,Cluster_Size,ID_Neighborhood
0,31994556,1,22,40.71815,-73.95834,Williamsburg,3456,1_Williamsburg
1,34181609,1,22,40.71808,-73.95839,Williamsburg,3456,1_Williamsburg
2,17415890,1,22,40.71813,-73.95822,Williamsburg,3456,1_Williamsburg
3,11960466,1,22,40.71214,-73.95837,Williamsburg,3456,1_Williamsburg
4,32191213,1,21,40.71211,-73.96276,Williamsburg,3456,1_Williamsburg
...,...,...,...,...,...,...,...,...
48890,35496842,1,0,40.88125,-73.84922,Williamsbridge,1,1_Williamsbridge
48891,27088022,1,0,40.59886,-74.13217,Willowbrook,1,1_Willowbrook
48892,23105516,1,0,40.89870,-73.86389,Woodlawn,1,1_Woodlawn
48893,1798271,1,0,40.53884,-74.19826,Woodrow,1,1_Woodrow


In [23]:
# Done